Main script elements:

STATIC VERSION
- Given a fixed DoD
- Main function
- Auxillary functions:
    - (greedy) insertion heuristic: in stead of looking for all possible insertions, look for a better insertion until you find no better one
        - initial solution creator > spatio-temporal clustering of requests?
        - feasibility check
        - incremental waiting time + travel time calculator
    - improvement heuristic
        - neighborhood creators (destroy&repair, swappers, L-opt moves, ...)
        - evaluate objective functions

In [20]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [21]:
# parameters

available_vehicles = 5
vehicle_capacity = 20
dod = 0 # static version = 0, dynamic version > 0
WT_penalty = 0 # how strongly is WT perceived <> TT?
TT_penalty = 0 

In [34]:
# NOTE: this is only 1 scenario and 1 direction!!
file = open("Data/requests.pkl", "rb")
Requests = pickle.load(file)

file_2 = open("Data/small_od_matrix.pkl", "rb")
od_matrix = pickle.load(file_2)

vehicles = {}
schedule_so_far = {}

In [46]:
od_pairs = set([i[0] for i in Requests])
od_pairs

{(1, 2), (1, 3), (2, 3), (3, 4), (3, 5), (4, 5)}

In [35]:
# Class: requests
# methods: select on destination, cluster/group on pickup time, assign issue time, ...

Requests

[((1, 2), 0, 0),
 ((1, 2), 28.407113207489235, 0),
 ((1, 2), 32.17669168539131, 0),
 ((1, 2), 40.236454807666185, 0),
 ((1, 2), 42.17632695554921, 0),
 ((1, 2), 50.50785170520504, 0),
 ((1, 2), 55.70797952881701, 0),
 ((1, 2), 58.66157588784098, 0),
 ((1, 3), 0, 0),
 ((1, 3), 0.1403039226465128, 0),
 ((1, 3), 0.20978743783730414, 0),
 ((1, 3), 0.21631632923156086, 0),
 ((1, 3), 0.4281202641242935, 0),
 ((1, 3), 0.5071587541090863, 0),
 ((1, 3), 1.0027513055631039, 0),
 ((1, 3), 1.0863795908753289, 0),
 ((1, 3), 1.1665300400841745, 0),
 ((1, 3), 1.3783385157453578, 0),
 ((1, 3), 1.6856287815563562, 0),
 ((1, 3), 1.9200114705257139, 0),
 ((1, 3), 2.129864580396628, 0),
 ((1, 3), 2.1916648507755148, 0),
 ((1, 3), 2.3473259136371643, 0),
 ((1, 3), 2.354725611587812, 0),
 ((1, 3), 2.410616036755492, 0),
 ((1, 3), 2.5607007024000072, 0),
 ((1, 3), 2.6223601718351692, 0),
 ((1, 3), 2.824815250699431, 0),
 ((1, 3), 3.0141667409527573, 0),
 ((1, 3), 3.0744490680552103, 0),
 ((1, 3), 3.266362590

In [36]:
# make this later a method of the class 'requests / request list'
def select_requests(requests, od):
    return [x for x in requests if x[0] == od]

select_requests(Requests, (1,3))


[((1, 3), 0, 0),
 ((1, 3), 0.1403039226465128, 0),
 ((1, 3), 0.20978743783730414, 0),
 ((1, 3), 0.21631632923156086, 0),
 ((1, 3), 0.4281202641242935, 0),
 ((1, 3), 0.5071587541090863, 0),
 ((1, 3), 1.0027513055631039, 0),
 ((1, 3), 1.0863795908753289, 0),
 ((1, 3), 1.1665300400841745, 0),
 ((1, 3), 1.3783385157453578, 0),
 ((1, 3), 1.6856287815563562, 0),
 ((1, 3), 1.9200114705257139, 0),
 ((1, 3), 2.129864580396628, 0),
 ((1, 3), 2.1916648507755148, 0),
 ((1, 3), 2.3473259136371643, 0),
 ((1, 3), 2.354725611587812, 0),
 ((1, 3), 2.410616036755492, 0),
 ((1, 3), 2.5607007024000072, 0),
 ((1, 3), 2.6223601718351692, 0),
 ((1, 3), 2.824815250699431, 0),
 ((1, 3), 3.0141667409527573, 0),
 ((1, 3), 3.0744490680552103, 0),
 ((1, 3), 3.266362590603292, 0),
 ((1, 3), 4.1817065240613065, 0),
 ((1, 3), 4.5109544924087395, 0),
 ((1, 3), 4.621483818658806, 0),
 ((1, 3), 5.31707927800999, 0),
 ((1, 3), 5.503083265658093, 0),
 ((1, 3), 5.533734367763384, 0),
 ((1, 3), 5.920013815727957, 0),
 ((1, 

In [55]:
# create clusters per od-pair = grouping together or request in time

def group_requests(requests, groupsize):
    requests_dict = {}
    for od in od_pairs:
        requests_dict[od] = [select_requests(requests, od)[x:x+groupsize] for x in 
                             range(0, len(select_requests(requests, od)), groupsize)]
        
    return requests_dict

In [60]:
Requests_grouped = group_requests(Requests, 10)
Requests_grouped

{(2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.044419555169702155, 0),
   ((2, 3), 0.04638482974003759, 0),
   ((2, 3), 0.06732852263625788, 0),
   ((2, 3), 0.3284634611871504, 0),
   ((2, 3), 0.5434003273570929, 0),
   ((2, 3), 1.0797676944847718, 0),
   ((2, 3), 1.7902979871508609, 0),
   ((2, 3), 1.8758914913921108, 0),
   ((2, 3), 2.133016617912408, 0)], [((2, 3), 3.057734012280939, 0),
   ((2, 3), 3.4053477211684284, 0),
   ((2, 3), 3.712514703541706, 0),
   ((2, 3), 4.272340033953835, 0),
   ((2, 3), 4.284389259114676, 0),
   ((2, 3), 4.474053310311324, 0),
   ((2, 3), 4.657700402980071, 0),
   ((2, 3), 4.7811534136736595, 0),
   ((2, 3), 5.4660177544063355, 0),
   ((2, 3), 5.614894233824923, 0)], [((2, 3), 6.264964380297658, 0),
   ((2, 3), 6.457997598099996, 0),
   ((2, 3), 7.0136947860661385, 0),
   ((2, 3), 7.098659543904845, 0),
   ((2, 3), 7.141846088118017, 0),
   ((2, 3), 7.59659450316839, 0),
   ((2, 3), 7.599313469928337, 0),
   ((2, 3), 7.654896755614335, 0),
   ((2, 3), 7

In [62]:
# how many groups of at most 20 requests are there?
for k in Requests_grouped:
   print(k, len(Requests_grouped[k]))

(2, 3) 23
(1, 3) 25
(4, 5) 1
(1, 2) 1
(3, 5) 1
(3, 4) 2


In [13]:
od_matrix

{(1, 1): 0.0,
 (1, 2): 0.12434830115445887,
 (1, 3): 0.125,
 (1, 4): 0.12434830115445887,
 (1, 5): 0.0,
 (2, 1): 0.12434830115445887,
 (2, 2): 0.0,
 (2, 3): 0.12434830115445887,
 (2, 4): 0.0,
 (2, 5): 0.12434830115445887,
 (3, 1): 0.125,
 (3, 2): 0.12434830115445887,
 (3, 3): 0.0,
 (3, 4): 0.12434830115445887,
 (3, 5): 0.125,
 (4, 1): 0.12434830115445887,
 (4, 2): 0.0,
 (4, 3): 0.12434830115445887,
 (4, 4): 0.0,
 (4, 5): 0.12434830115445887,
 (5, 1): 0.0,
 (5, 2): 0.12434830115445887,
 (5, 3): 0.125,
 (5, 4): 0.12434830115445887,
 (5, 5): 0.0}

Insertion strategy: momentarily focussed on the capacity constraint (i.e. trying to fill vehicles), not so much on the time constraint

1. Divide requests into chunks: 
 * (a) of at most equal size (e.g. 10) 
 * (b) of optimal size (according to jenk's borders)
2. Select a first chunk of travellers, who want to travel from the origin to the farthest point, and fill a bus with them (almost) 
    * If this bus is ride is full, then this schedule is complete.
    * If there is room left, then add passengers in between at each visited stop.


In [64]:
tocity_keys = [(1,2),(2,3),(1,3)]
toterminal_keys = [(3,4),(3,5),(4,5)]

filterByKey = lambda keys: {x: Requests_grouped[x] for x in keys}
tocity_requests = filterByKey(tocity_keys)
toterminal_requests = filterByKey(toterminal_keys)

tocity_requests

{(1,
  2): [[((1, 2), 0, 0),
   ((1, 2), 28.407113207489235, 0),
   ((1, 2), 32.17669168539131, 0),
   ((1, 2), 40.236454807666185, 0),
   ((1, 2), 42.17632695554921, 0),
   ((1, 2), 50.50785170520504, 0),
   ((1, 2), 55.70797952881701, 0),
   ((1, 2), 58.66157588784098, 0)]],
 (2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.044419555169702155, 0),
   ((2, 3), 0.04638482974003759, 0),
   ((2, 3), 0.06732852263625788, 0),
   ((2, 3), 0.3284634611871504, 0),
   ((2, 3), 0.5434003273570929, 0),
   ((2, 3), 1.0797676944847718, 0),
   ((2, 3), 1.7902979871508609, 0),
   ((2, 3), 1.8758914913921108, 0),
   ((2, 3), 2.133016617912408, 0)], [((2, 3), 3.057734012280939, 0),
   ((2, 3), 3.4053477211684284, 0),
   ((2, 3), 3.712514703541706, 0),
   ((2, 3), 4.272340033953835, 0),
   ((2, 3), 4.284389259114676, 0),
   ((2, 3), 4.474053310311324, 0),
   ((2, 3), 4.657700402980071, 0),
   ((2, 3), 4.7811534136736595, 0),
   ((2, 3), 5.4660177544063355, 0),
   ((2, 3), 5.614894233824923, 0)], [((2, 3), 6.2

In [65]:
def calc_dep_time(request_group):
    return request_group[-1]

In [49]:
def create_initial_solution(request_dict, start=1, end=3, nb_of_vehicles=5, max_capacity=20, 
                            vehicles_schedule=None, current_vehicle=1):
    
    if vehicles_schedule is None:
        vehicles_schedule = {}
        
    if len(request_dict) == 0 or current_vehicle > nb_of_vehicles:
        return vehicles_schedule
    
    # always select the next empty vehicle in line
    
    for rg in request_dict[(start, end)]: # rg = a request group
        dep_time = calc_dep_time(rg)
        vehicles_schedule[current_veh] = {}
        # this should be a list where you add groups
        vehicles_schedule[current_veh][dep_time] = rg
        
        #make this an auxillary function
        available_cap = max_capacity - 
        
        load += sum(rg) # to be inserted elsewhere!
        
        # sum(rg) should be replaced by the total capacity, not only of the most recently added group
        if sum(rg) != max_capacity:
            start = end-1 #look for possible insertions of the passengers bounded one stop before the end
            # [1] --> 2 --> 3 --> [4] --> [5]
            # e.g. if here you research whether you can transport passengers from 4 --> 5,
            # AND if this is feasible, you might as well check if transporting pax from 1 --> 4 is also feasible
            
            # add another subdestination
        else:
            current_vehicle += 1
            # remove from request_dict the recently assigned requests
            return create_intial_solution(request_dict, nb_of_vehicles, max_capacity, vehicles_schedule)
            # rerun the solution

            

IndentationError: expected an indented block (<ipython-input-49-a29bce8c19a7>, line 18)

In [7]:
def insert_passenger(requests, vehicles, schedule_so_far):
    
    if len(requests) == 0:
        return schedule_so_far
    
    else:
        for r in requests:
            greedy_insert()
            requests.pop(r)
            
            insert_passenger(requests, vehicles, schedule_so_far)
    
    return None